# Srock Strategy - Buy & Hold 0003

## 1. Import Library

In [6]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [7]:
sd = '2026-01-06'
ed = '2026-01-14'

In [8]:
instrument_list = dai.query(f"""
SELECT DISTINCT instrument 
FROM cn_stock_dragon_list 
WHERE date = '{sd}' 
AND instrument NOT LIKE '%BJ%'
ORDER BY close
LIMIT 5
""").df().instrument.to_list()

In [9]:
sql = f"""
WITH
data_buy AS (
    WITH
    data_min_date AS (
        SELECT
            date,
            instrument,
            MIN(date) OVER (PARTITION BY instrument) AS _min_date,
        FROM cn_stock_bar1d
        WHERE instrument IN {str(tuple(instrument_list))}
        QUALIFY (date = _min_date)
        ORDER by date, instrument
    )
    SELECT
        date,
        instrument,
        1 / SUM(1) OVER (PARTITION BY 1) AS position
    FROM data_min_date
),
data_sell AS (
    SELECT
        date,
        instrument,
        MAX(date) OVER (PARTITION BY instrument) AS _max_date,
        0 AS position,
    FROM cn_stock_bar1d
    WHERE instrument IN {str(tuple(instrument_list))}
    QUALIFY (date = _max_date)
    ORDER by date, instrument
)
SELECT * 
FROM data_buy 
UNION
SELECT * 
FROM data_sell
ORDER BY date, instrument
"""

In [10]:
df = dai.query(sql, filters = {"date":[sd, ed]}).df()
df

,date,instrument,position
0,2026-01-06,000981.SZ,0.2
1,2026-01-06,300379.SZ,0.2
2,2026-01-06,600370.SH,0.2
3,2026-01-06,600691.SH,0.2
4,2026-01-06,600936.SH,0.2
5,2026-01-14,000981.SZ,0.0
6,2026-01-14,300379.SZ,0.0
7,2026-01-14,600370.SH,0.0
8,2026-01-14,600691.SH,0.0
9,2026-01-14,600936.SH,0.0


## 3. Backtest Strategy

In [11]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 500000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 14:14:49] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 14:14:49] [info     ] 2026-01-06, 2026-01-14, , , instruments=5
[2026-02-18 14:14:49] [info     ] bigtrader module V2.2.0
[2026-02-18 14:14:49] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 14:14:49] [info     ] backtest done, raw_perf_ds:dai.DataSource("_e3ed2fedba93429589c283473c52d8ac")


[2026-02-18 14:14:49] [info     ] bigtrader.v34 运行完成 [0.812s].
